# Temporary Jupyter Notebook for SCRIPT 4

## Rachel's Part:

In [1]:
import numpy as np
import pandas as pd
import altair as alt

In [2]:
from hashlib import sha1

import matplotlib.pyplot as plt
from IPython.display import HTML

from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.preprocessing import (
    FunctionTransformer,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    normalize,
    scale,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)

from sklearn.model_selection import (
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC

In [3]:
# set max column and max row display of dataframes
pd.set_option("display.max_colwidth", 200)
pd.set_option('display.max_rows', 50)

### Import processed data (not sure how to do this...)

We can delete the next cell after downloading processed data:

In [4]:
# Packages necessary for importing data (from a zip file containing 2 dataset CSVs)
import requests, zipfile
from urllib.request import urlopen
from io import BytesIO

zip_file_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip"
zip_file_load = urlopen(zip_file_url)
zipinmemory = BytesIO(zip_file_load.read())
zip_file = zipfile.ZipFile(zipinmemory)

# Only load the first file in the zip folder
diabetes_csv = pd.read_csv(zip_file.open(zip_file.namelist()[0]))

# Change `readmitted` target column to binary "YES" or "NO" values if admitted or not.
pattern = r'[<>]30'
diabetes_csv["readmitted"] = diabetes_csv["readmitted"].str.replace(pattern,"YES",regex = True)

# Convert any ? to na
diabetes_csv = diabetes_csv.replace("?", np.NaN)

# Drop any rows with na
diabetes_clean = diabetes_csv.dropna()

# Drop columns not useful to answering our question
diabetes_clean = diabetes_csv.drop(columns = ["encounter_id", "patient_nbr", "race", "weight", "payer_code", "medical_specialty", "examide", "citoglipton"])

diabetes_clean.head(10)

,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Female,[0-10),6,25,1,1,41,0,1,0,...,No,No,No,No,No,No,No,No,No,NO
1,Female,[10-20),1,1,7,3,59,0,18,0,...,No,Up,No,No,No,No,No,Ch,Yes,YES
2,Female,[20-30),1,1,7,2,11,5,13,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,Male,[30-40),1,1,7,2,44,1,16,0,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Male,[40-50),1,1,7,1,51,0,8,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,Male,[50-60),2,1,2,3,31,6,16,0,...,No,Steady,No,No,No,No,No,No,Yes,YES
6,Male,[60-70),3,1,2,4,70,1,21,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,Male,[70-80),1,1,7,5,73,0,12,0,...,No,No,No,No,No,No,No,No,Yes,YES
8,Female,[80-90),2,1,4,13,68,2,28,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,Female,[90-100),3,3,4,12,33,3,18,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


### Split Data into Training and Testing

In [5]:
# Take a random and representative sample of our diabetes dataset to apply data analysis to

diabetes_subset = diabetes_clean.sample(n = 10_000)
diabetes_subset

,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
69020,Female,[60-70),1,22,7,6,72,1,22,0,...,No,Down,No,No,No,No,No,Ch,Yes,YES
38138,Female,[50-60),2,1,7,7,1,3,5,0,...,No,Steady,No,No,No,No,No,No,Yes,YES
7541,Male,[80-90),3,6,1,8,37,6,51,0,...,No,No,No,No,No,No,No,Ch,Yes,NO
69531,Male,[60-70),5,6,7,6,40,0,10,0,...,No,No,No,No,No,No,No,Ch,Yes,YES
60964,Female,[80-90),1,1,7,2,41,4,17,0,...,No,Steady,No,No,No,No,No,No,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19585,Female,[80-90),2,22,1,3,15,2,11,0,...,No,No,No,No,No,No,No,No,Yes,YES
98504,Male,[30-40),3,1,4,1,39,6,11,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
73576,Male,[70-80),1,1,7,3,39,0,4,1,...,No,No,No,No,No,No,No,No,No,NO
70510,Female,[70-80),3,3,1,4,10,3,28,0,...,No,No,No,No,No,No,No,Ch,Yes,NO


In [6]:
# Change positive and negative labels of readmitted target column to 0 (not readmitted) and 1 (readmitted)

from sklearn.preprocessing import label_binarize
encoded_column_vector = label_binarize(diabetes_subset['readmitted'], classes=['NO','YES'])
encoded_labels = np.ravel(encoded_column_vector)

diabetes_subset["readmitted"] = encoded_labels

In [7]:
# Split the data into training (0.8) and testing (0.2)
train_df, test_df = train_test_split(diabetes_subset, test_size=0.2, random_state=123)

# Split the data into X and Y
X_train, y_train = train_df.drop(columns=["readmitted"]), train_df["readmitted"] 
X_test, y_test = test_df.drop(columns=["readmitted"]), test_df["readmitted"]

### Label features

In [8]:
# categorical features - OneHotEncoding
# numeric features - StandardScaler
# ordinal features - OrdinalEncoding
categorical_features = ["age", "diag_1", "diag_2", "diag_3", "max_glu_serum", "A1Cresult", "metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"]
numeric_features = ["admission_type_id", "discharge_disposition_id", "admission_source_id", "time_in_hospital", "num_lab_procedures", "num_procedures", "num_medications", "number_outpatient", "number_emergency", "number_inpatient", "number_diagnoses" ]
ordinal_features = ["gender", "change", "diabetesMed"]
target_feature = "readmitted"

### Create transformers and preprocesser pipeline

In [9]:
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")), 
            ("scaler", StandardScaler()),
    ]
)

ordinal_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("ordinal", OrdinalEncoder()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("ord", ordinal_transformer, ordinal_features)
    ],
    #remainder = "passthrough"
)

In [10]:
preprocessor.fit(X_train, y_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['admission_type_id',
                                  'discharge_disposition_id',
                                  'admission_source_id', 'time_in_hospital',
                                  'num_lab_procedures', 'num_procedures',
                                  'num_medications', 'number_outpatient',
                                  'number_emergency', 'number_inpatient',
                                  'number_diagnoses...
                                  'tolbutamide', 'pioglitazone',
                                  'rosiglitazone', 'acarbose', 'miglitol',
                                  'troglitazone', 'tolazamide',
                                  'glyburide-metformin', '

### Create model pipelines and test out different models (RBF SVM and LR) against DummyClassifier baseline

In [19]:
# Create an empty dictionary to store results
results_dict = {}

In [23]:
# Code adapted from MDS 571 - Lab 4
def store_results(classifier_name, scores, results_dict):
    """
    Stores mean scores from cross_validate in results_dict for
    the given model model_name.

    Parameters
    ----------
    model_name :
        scikit-learn classification model
    scores : dict
        object return by `cross_validate`
    results_dict: dict
        dictionary to store results

    Returns
    ----------
        None

    """
    # test cases for store_results function
    assert type(classifier_name) == str
    assert type(scores) == dict
    
    results_dict[classifier_name] = {
        "fit_time": "{:0.4f}".format(np.mean(scores["fit_time"])),
        "score_time": "{:0.4f}".format(np.mean(scores["score_time"])),
        "test_accuracy": "{:0.4f}".format(np.mean(scores["test_accuracy"])),
        "train_accuracy": "{:0.4f}".format(np.mean(scores["train_accuracy"])),
        "test_f1": "{:0.4f}".format(np.mean(scores["test_f1"])),
        "train_f1": "{:0.4f}".format(np.mean(scores["train_f1"])),
        "test_recall": "{:0.4f}".format(np.mean(scores["test_recall"])),
        "train_recall": "{:0.4f}".format(np.mean(scores["train_recall"])),
        "test_precision": "{:0.4f}".format(np.mean(scores["test_precision"])),
        "train_precision": "{:0.4f}".format(np.mean(scores["train_precision"])),
        "test_average_precision": "{:0.4f}".format(np.mean(scores["test_precision"])),
        "train_average_precision": "{:0.4f}".format(np.mean(scores["train_precision"])),
        "test_roc_auc": "{:0.4f}".format(np.mean(scores["test_roc_auc"])),
        "train_roc_auc": "{:0.4f}".format(np.mean(scores["train_roc_auc"])),
    }

In [24]:
# Test 3 models against baseline DummyClassifier
classifiers = {
    "Dummy Classifier" : DummyClassifier(strategy = "most_frequent"),
    "RBF SVM": SVC(),
    "Logistic Regression": LogisticRegression(),
    "Logistic Regression (balanced)": LogisticRegression(class_weight="balanced"),
}

In [25]:
# ignore warnings, DummyClassifier will output many 0s for scores but this is correct
#import warnings
#warnings.filterwarnings('ignore')

scoring = ["accuracy", "f1", "recall", "precision", "average_precision", "roc_auc"]

for classifier_name, classifier in classifiers.items():
    pipe = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", classifier)])
    scores = cross_validate(pipe, X_train, y_train, return_train_score=True, scoring = scoring)
    store_results(classifier_name, scores, results_dict)

results_dict = pd.DataFrame(results_dict).T

results_dict

/opt/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precis

,fit_time,score_time,test_accuracy,train_accuracy,test_f1,train_f1,test_recall,train_recall,test_precision,train_precision,test_average_precision,train_average_precision,test_roc_auc,train_roc_auc
Dummy Classifier,0.1030,0.0727,0.5377,0.5378,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5000,0.5000
RBF SVM,5.3389,1.9746,0.6127,0.6691,0.4799,0.5602,0.3864,0.4559,0.6330,0.7265,0.6330,0.7265,0.6420,0.7635
Logistic Regression,0.2915,0.0508,0.5930,0.6910,0.5170,0.6360,0.4711,0.5838,0.5728,0.6983,0.5728,0.6983,0.6145,0.7588
Logistic Regression (balanced),0.2759,0.0558,0.5879,0.6918,0.5503,0.6670,0.5454,0.6679,0.5552,0.6663,0.5552,0.6663,0.6151,0.7582


According to the results it looks like Logistic Regression (balanced) had the highest training accuracy and f1 scores.

### Continuing work with Logistic Regression (balanced) pipeline

In [15]:
lr_bal_pipe = Pipeline(steps=[("preprocessor", preprocessor), ("lr", LogisticRegression(class_weight="balanced"))])

### Hyperparameter Optimization with Logistic Regression (balanced)

### Hyperparameter Optimization results (confusion matrix, precision-recall curve, AUC curve)

### Use Logistic Regression (balanced) model and best hyperparameters on test set

### Top Coefficients of Best Indicator Features

### EXTRA: Find Test Set With Most Predictive Readmission Outcome vs. No Readmission Outcome